In [ ]:
import numpy as np
import os
import pandas as pd
from graph_traffic.custom_funcs import  make_stable_values_null, rows_no_change
from graph_traffic.config import data_path

traffic_path = os.path.join(data_path, "03-by-location", "traffic")
meteo_path = os.path.join(data_path, "03-by-location", "meteo")


tmagns = ['intensidad', 'ocupacion', 'vmed']
mmagns = ['temperatura', 'humedad_relativa', 'presion_barometrica', 'radiacion_solar',
          'precipitacion', 'dir_viento', 'velocidad_viento']

mapping = pd.read_csv(os.path.join(data_path, '03-by-location', 'id_mapping.csv'))

traffic_files = os.listdir(traffic_path)
n_traffic_files = len(traffic_files)

for i, t_file in enumerate(traffic_files):
    print(f"{i+1}/{n_traffic_files}", end="\r")
    t_location = t_file.split(".")[0]
    # id de la estación meteorológica correspondiente a cada magnitud
    ids_m = mapping[mapping.id_t == int(t_location)].iloc[0][[f'id_{magn}' for magn in mmagns]].astype(int)

    # Leer los datos de tráfico
    dft = pd.read_csv(f'{traffic_path}/{t_location}.csv', parse_dates=['fecha'], index_col='fecha')
    if dft.empty:
        continue
    # Para cada estación, leemos los datos necesarios
    dfm = {estacion: pd.read_csv(f'{meteo_path}/estacion-{estacion:.0f}.csv', parse_dates=['fecha'], index_col='fecha') for estacion in ids_m.unique()}

    # Si hay más de 4 filas sin cambio, damos el valor por nulo
    dft[tmagns] = dft[tmagns].apply(make_stable_values_null, nrows=4)
    for estacion, dfmi in dfm.items():
        nm = dfmi[mmagns].apply(rows_no_change)
        for m in mmagns:
            if m in ['precipitacion', 'radiacion_solar', 'presion_barometrica']:
                continue
            dfmi[m] = np.where((nm[m]>4) & (dfmi[m]!=0), np.nan, dfmi[m])
        dfm[estacion] = dfmi

    # Hacer el merge de todas las variables meteorológicas con el tráfico
    df = dft
    for m in mmagns:
        df = df.merge(dfm[ids_m[f"id_{m}"]][[m]],
                     left_index=True, right_index=True,
                    how='left')

    del dft, dfm
    df = df.sort_index()
    df[mmagns] = df[mmagns].interpolate(method="linear", limit=4)

    df.to_csv(f"{data_path}/04-traffic-meteo-merged/{t_location}.csv")

df.head()

In [6]:
df

,id,intensidad,ocupacion,vmed,temperatura,humedad_relativa,presion_barometrica,radiacion_solar,precipitacion,dir_viento,velocidad_viento
fecha,,,,,,,,,,,
2019-02-01 00:00:00,9999,10.0,NaN,NaN,6.000,96.00,929.00,0.0,0.300,219.00,2.0500
2019-02-01 00:15:00,9999,10.0,NaN,NaN,6.000,96.25,928.75,0.0,0.500,217.25,2.0675
2019-02-01 00:30:00,9999,9.0,NaN,NaN,6.000,96.50,928.50,0.0,0.700,215.50,2.0850
2019-02-01 00:45:00,9999,11.0,NaN,NaN,6.000,96.75,928.25,0.0,0.900,213.75,2.1025
2019-02-01 01:00:00,9999,10.0,NaN,NaN,6.000,97.00,928.00,0.0,1.100,212.00,2.1200
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 22:15:00,9999,14.0,2.0,NaN,2.425,93.00,937.00,0.0,0.225,211.50,1.4550
2020-12-31 22:30:00,9999,14.0,1.0,NaN,2.350,93.00,937.00,0.0,0.450,216.00,1.3300
2020-12-31 22:45:00,9999,6.0,0.0,NaN,2.275,93.00,937.00,0.0,0.675,220.50,1.2050
